**Script que genera SCI y CCI de la última imágen disponible en el catálogo de GEE.**

SCI: Snow Cover Index

CCI: Cloud Cover Index

Este script tiene el proposito de automatizar el proceso de exportación del promedio mensual de SCI y CCI. 

To automate you need to sign-in using a service account
1. Create or Select a Google Cloud Project
2. Enable the project for 'Earth Engine API'
3. Create a service Account
    3.1 Grant role: Earth Engine Resource Writer (Avoiding Manage role to limit risk)
4. Create keys for service account (json file). Store this file securely
5. Activate the service account for GEE here (https://signup.earthengine.google.com/#!/service_accounts)
6. test sign in from python.

In [1]:
# libraries
import ee
import geemap
from datetime import datetime
import pytz
import json
import pprint
import pathlib
from time import sleep
from yaspin import yaspin



In [2]:
# Custom libraries
import nieve_sequia_auto as nsa

In [3]:
# Constants
GEE_PATHPREFIX = 'projects/earthengine-legacy/assets/'
DPA_REGIONES_NACIONALES = 'users/proyectosequiateleamb/Regiones/DPA_regiones_nacional'
SAVED_IMAGES_PATH = 'users/proyectosequiateleamb/Nieves/Raster_SCI_CCI'
MODIS_MIN_MONTH = '2000-03'
UTC_TZ=pytz.timezone('UTC')

In [4]:
# Replace Credentials with your service account
service_account = "nieves-test@ee-chompitest.iam.gserviceaccount.com"
credentials = ee.ServiceAccountCredentials(service_account, 'ee-chompitest-ccb9a3676966.json')
ee.Initialize(credentials)

In [5]:
# Get feature collection for DPA territorio Nacional if found
try: 
    if nsa.check_asset_exists(DPA_REGIONES_NACIONALES):
        ee_territorio_nacional = ee.FeatureCollection(DPA_REGIONES_NACIONALES)

except:
    pass

Searching for: users/proyectosequiateleamb/Regiones/DPA_regiones_nacional
Found 2 assets
Asset successfully found: True


In [6]:
# Get MODIS image Collection and take date from the last image.
# Remove images from current month to avoid incomplete months

ee_MODIS_collection = (ee.ImageCollection('MODIS/006/MOD10A1'))
ee_MODIS_collection = ee_MODIS_collection.filterDate(MODIS_MIN_MONTH, nsa.current_year_month())

In [7]:
# Get distinct months in MODIS collection
MODIS_distinct_months = nsa.get_ic_distinct_months(ee_MODIS_collection)


Total images in collection: 8273
Distinct months in collection: 273
first month: 2000-03-01
last month: 2022-12-01


In [8]:
# Get list of months of images already saved 
saved_assets=nsa.get_asset_list(SAVED_IMAGES_PATH)
saved_assets_months = [date[-7:]+'-01' for date in saved_assets]
saved_assets_months.sort(reverse=True)

print(f"Total images saved in folder: {len(saved_assets_months)}")
print(f"first month saved: {saved_assets_months[-1]}")
print(f"last month saved: {saved_assets_months[0]}")
# mising_months = [date for date in MODIS_distinct_months if date not in saved_assets_months]

Total images saved in folder: 274
first month saved: 2000-02-01
last month saved: 2022-11-01


In [9]:
# Check if last available month in MODIS is already saved

last_saved = saved_assets_months[0]
last_available = MODIS_distinct_months[0]
if(last_saved == last_available):
    print(f"Last available month is already saved: {last_available}")
else:
    months_to_save=[last_available]
    print(f"Pending months: {months_to_save}")



Pending months: ['2022-12-01']


In [10]:

# Calcular y seleccionar banda de nieve binario y nubes en la coleccion
ee_snow_cloud_collection = ee_MODIS_collection.map(nsa.snow_cloud_mask).select('SCI', 'CCI');


In [11]:
# Calculate mean for last month in collection
if len(months_to_save)>=1:
    ee_monthly_snow_cloud_collection = nsa.imagecollection_monthly_mean(months_to_save, ee_snow_cloud_collection)


Processing: 2022-12-01 00:00:00+00:00 - 2023-01-01 00:00:00+00:00


In [12]:
# Sort and get last image only in case there's more than one month processed
ee_monthly_snow_cloud_collection = ee_monthly_snow_cloud_collection.sort(
        prop='system:time_start', 
        opt_ascending=False)

ee_last_snow_cloud_image = ee_monthly_snow_cloud_collection.first()

In [13]:
last_image_info = ee_last_snow_cloud_image.getInfo()
pprint.pprint(last_image_info)

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 100,
                          'min': 0,
                          'precision': 'double',
                          'type': 'PixelType'},
            'id': 'SCI'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 100,
                          'min': 0,
                          'precision': 'double',
                          'type': 'PixelType'},
            'id': 'CCI'}],
 'properties': {'month': 12,
                'system:index': '0',
                'system:time_start': '2022-12',
                'year': 2022},
 'type': 'Image'}


In [14]:
# Exportar última imagen de la coleccion al Asset GEE
property = ee_last_snow_cloud_image.get('system:time_start').getInfo()
image_name = 'MOD10A1_SCI_CCI_' + property


task = ee.batch.Export.image.toAsset(**{
  'image': ee_last_snow_cloud_image,
  'description': image_name,
  #'assetId': pathlib.Path(SAVED_IMAGES_PATH, image_name).as_posix()
  'assetId': 'projects/ee-chompitest/assets/snow/Raster_SCI_CCI/'+image_name,
  'scale': 500,
  'region': ee_territorio_nacional.geometry(),
})


In [15]:
task.start()

In [18]:
nsa.check_gee_task_status(task, wait_time=30)

status: FAILED
Cannot overwrite asset 'projects/ee-chompitest/assets/snow/Raster_SCI_CCI/MOD10A1_SCI_CCI_2022-12'.
